In [1]:
# Displaying graphs
import plotly.io as pio
pio.renderers.default = 'iframe'

import pandas as pd
import numpy as np
from PIL import Image
from urllib.request import urlopen
import io
from wordcloud import WordCloud
from matplotlib.colors import LinearSegmentedColormap
import base64
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
# Import dataframe
united_df = pd.read_csv('Steam Store Games (Clean dataset)/steam.csv')

# rating_difference is needed to determine popular games that users liked or did not like
united_df['rating_difference'] = united_df['positive_ratings'] - united_df['negative_ratings']

In [3]:
# Downloading an image from wikipedia
fd = urlopen("https://upload.wikimedia.org/wikipedia/commons/8/87/New_Steam_Logo_with_name.jpg")
image_file = io.BytesIO(fd.read())
im = Image.open(image_file)

# Removing bad games
malo_df = united_df.drop(united_df[united_df['rating_difference'] <= 10].index).reset_index(drop=True)

# The size of the lettering will depend on the rating
dictionary = dict(zip(malo_df.name, malo_df.rating_difference))

# cmap creation
colors = ["#000000", "#111111", "#101010", "#121212", "#212121", "#222222"]
cmap = LinearSegmentedColormap.from_list("mycmap", colors)

In [4]:
# WordCloud creation
cloud_full = WordCloud(background_color="#FFFFFF", mask=np.array(im),
                       min_font_size=2, max_font_size=30, colormap = cmap,
                       max_words=2000).generate_from_frequencies(dictionary)
cloud_image_full = cloud_full.to_image()

In [5]:
# Cropping the picture to highlight the logo
im_square = im.crop((0, 0, 695, 695))

# Creation of a WordCloud with only logo
cloud_square = WordCloud(background_color="#FFFFFF", mask=np.array(im_square),
                         min_font_size=2, max_font_size=30, colormap = cmap,
                         max_words=2000).generate_from_frequencies(dictionary)
cloud_image_square = cloud_square.to_image()

In [6]:
# Function for displaying a picture with plotly
def plotly_plot(img, img_width, img_height):
    fig = go.Figure()
    scale_factor = 0.5
    # Add invisible scatter trace.
    # This trace is added to help the autoresize logic work.
    fig.add_trace(
        go.Scatter(
            x=[0, img_width * scale_factor],
            y=[0, img_height * scale_factor],
            mode="markers",
            marker_opacity=0)
    )

    # Configure axes
    fig.update_xaxes(
        visible=False,
        range=[0, img_width * scale_factor])

    fig.update_yaxes(
        visible=False,
        range=[0, img_height * scale_factor],
        # the scaleanchor attribute ensures that the aspect ratio stays constant
        scaleanchor="x")

    # Add image
    fig.add_layout_image(
        dict(
            x=0,
            sizex=img_width * scale_factor,
            y=img_height * scale_factor,
            sizey=img_height * scale_factor,
            xref="x",
            yref="y",
            opacity=1.0,
            layer="below",
            sizing="stretch",
            source=img)
    )

    # Configure other layout
    fig.update_layout(
        width=img_width * scale_factor,
        height=img_height * scale_factor,
        margin={"l": 0, "r": 0, "t": 0, "b": 0})
    
    return fig

In [7]:
# Displaying a picture
plotly_plot(img = cloud_image_full,
            img_width = 2172, img_height = 695).show(config={'doubleClick': 'reset'})

In [8]:
# Displaying a picture
plotly_plot(img = cloud_image_square,
            img_width = 695, img_height = 695).show(config={'doubleClick': 'reset'})

In [21]:
# Saving pictures
cloud_image_full.save('cloud_image_full.png')
cloud_image_square.save('cloud_image_square.png')